In [1]:
import cadquery as cq
from jupyter_cadquery import (
    PartGroup, Part, Edges, Faces, Vertices, show,
    close_viewer, close_viewers, get_viewer, open_viewer, set_defaults, get_defaults
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay, reset_replay

set_defaults(axes=True, timeit=False)

cv = open_viewer("Examples", cad_width=640, height=480, glass=True)

enable_replay(False, False)
show_object = replay

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay


![Pico Diagram](Pico-Diagram.webp)


In [3]:
wall_thickness = 3

pico_width = 21
pico_length = 51
pico_height = 13

base_padding = 2

base_width = pico_width + base_padding + wall_thickness
base_length = pico_length + base_padding + wall_thickness

circles_width = 11.4
circles_length = 47
circles_outer = 2
circles_inner = 1
circles_height = 2

usb_width = 8


In [4]:
oshell = (
    cq.Workplane("XY")
    .box(base_length, base_width, pico_height)
    .edges("|Z").fillet(2)
)

ishell = (
    oshell.faces("<Z")
    .workplane(wall_thickness, True)
    .rect(pico_length+base_padding, pico_width+base_padding)
    .extrude(pico_height, False)
)

oshell = oshell.cut(ishell)

# Add circles
thick_circles = (
    oshell
    .faces("<Z")
    .workplane(wall_thickness, True)
    .rect(circles_length, circles_width, forConstruction=True)
    .vertices()
    .circle(circles_outer)
    .extrude(circles_height)
)

thin_circles = (
    oshell
    .faces("<Z")
    .workplane(wall_thickness, True)
    .rect(circles_length, circles_width, forConstruction=True)
    .vertices()
    .circle(circles_inner)
    .extrude(circles_height*2)           
)

box = oshell.union(thick_circles).union(thin_circles)

# USB
usb = (
    box
    .faces(">X")
    .workplane()
    .move(0, wall_thickness)
    .rect(usb_width*1.1, 10)
    .cutThruAll()
)

result = usb



result

In [5]:
cq.exporters.export(result, 'pico-base.stl')


In [80]:
relay_height = 20
relay_width = 18
relay_length = 44 # Increased slightly


oshell = (
    cq.Workplane("XY")
    .box(base_length, base_width, relay_height)
    .edges("|Z").fillet(2)
)

ishell = (
    oshell.faces("<Z")
    .workplane(wall_thickness*3, True)
    .rect(pico_length+base_padding, pico_width+base_padding)
    .extrude(relay_height, False)
)
    
oshell = oshell.cut(ishell)

relay = (
    oshell.faces("<Z")
    .workplane(wall_thickness*3, True)
    .move(3, 0)
    .rect(relay_length, relay_width)
    .extrude(2-wall_thickness*3, False)
)
    
oshell = oshell.cut(relay)

base = (
    oshell.faces("<Z")
    .rect(pico_length+base_padding-1, pico_width+base_padding-1)
    .extrude(-2, False)
)

oshell = oshell.union(base)

oshell = (
    oshell.faces("<Z")
    .rect(10, 10)
    .cutThruAll()
)

oshell = (
    oshell.faces("<Z")
    .workplane(0, True)
    .move(18, 0)
    .rect(10, 10)
    .cutThruAll()
)

oshell = (
    oshell.faces("<Z")
    .workplane(0, True)
    .move(-18, 0)
    .rect(10, 10)
    .cutThruAll()
)

oshell = (
    oshell.faces("<X")
    .workplane()
    .move(0, 12)
    .circle(6)
    .cutThruAll()

    
)

oshell


In [81]:
cq.exporters.export(oshell, 'relay-base.stl')


In [9]:
import cadquery as cq

w, l, h = 50, 20, 15
lsnap = 20
rsnap = 0.5

s = cq.Sketch().circle(rsnap)

b = (
    cq.Workplane()
    .box(w, l, h)
    .faces("<Y")
    .vertices("<X and >Z")
    .workplane(centerOption="CenterOfMass", offset=-(l + lsnap) / 2)
    .center(0, -2.0)
    .placeSketch(s)
    .extrude(lsnap, combine="s")
)
b

In [4]:
b